# Module 02: Prompt Engineering for Retail/eCommerce - SOLUTION

This notebook contains complete solutions to all exercises in Module 02.

In [1]:
import os
import json
import getpass
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, FewShotPromptTemplate
from dotenv import load_dotenv

load_dotenv()

# Prompt for API key only if not present in environment variables after loading .env
if not os.environ.get("AZURE_OPENAI_API_KEY"):
    os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass("Enter your Azure OpenAI API key: ")

# Initialize Azure OpenAI model
model = AzureChatOpenAI(
    azure_endpoint="https://aoi-ext-eus-aiml-profx-01.openai.azure.com/",
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    model="gpt-4o",
    api_version="2024-12-01-preview",
    temperature=0.7,
    top_p=1.0,
)

# Load data
with open('../../data/walmart_data.json', 'r') as f:
    walmart_data = json.load(f)

def get_completion(prompt, temperature=0.7):
    """Get completion from Azure OpenAI using LangChain"""
    temp_model = AzureChatOpenAI(
        azure_endpoint="https://aoi-ext-eus-aiml-profx-01.openai.azure.com/",
        api_key=os.environ["AZURE_OPENAI_API_KEY"],
        model="gpt-4o",
        api_version="2024-12-01-preview",
        temperature=temperature,
        top_p=1.0,
    )
    response = temp_model.invoke(prompt)
    return response.content

def get_completion_with_top_p(prompt, top_p=1.0, temperature=1.0):
    """Get completion with custom top_p from Azure OpenAI"""
    temp_model = AzureChatOpenAI(
        azure_endpoint="https://aoi-ext-eus-aiml-profx-01.openai.azure.com/",
        api_key=os.environ["AZURE_OPENAI_API_KEY"],
        model="gpt-4o",
        api_version="2024-12-01-preview",
        temperature=temperature,
        top_p=top_p,
    )
    response = temp_model.invoke(prompt)
    return response.content

## Exercise 1 Solution: Improved Review Analysis Prompt

In [2]:
# Get some reviews for the Samsung TV
tv_reviews = [r for r in walmart_data['customer_reviews'] if r['product_id'] == 'P001']

# SOLUTION: Improved prompt with clear role, structure, and requirements
improved_prompt = f"""You are a senior customer insights analyst for Walmart's eCommerce platform.
Your expertise is in extracting actionable insights from customer feedback to improve products and services.

Task: Analyze the following customer reviews for {walmart_data['products'][0]['name']}

Reviews:
{chr(10).join([f"{i+1}. Rating: {r['rating']}/5 - {r['review_text']}" for i, r in enumerate(tv_reviews)])}

Please provide your analysis in the following format:

1. **Overall Sentiment**: (Positive/Negative/Mixed) with percentage breakdown
2. **Common Praise**: Top 3 features customers love
3. **Common Complaints**: Top 3 issues or concerns
4. **Product Team Recommendations**: 2-3 specific actionable improvements
5. **Marketing Opportunities**: Which positive aspects should be highlighted in marketing?

Keep your analysis concise and data-driven."""

print("IMPROVED PROMPT:")
print(improved_prompt)
print("\n" + "="*80 + "\n")
print("AI RESPONSE:")
print(get_completion(improved_prompt, temperature=0.3))

IMPROVED PROMPT:
You are a senior customer insights analyst for Walmart's eCommerce platform.
Your expertise is in extracting actionable insights from customer feedback to improve products and services.

Task: Analyze the following customer reviews for Samsung 55-inch 4K Smart TV

Reviews:
1. Rating: 5/5 - Amazing picture quality! The 4K resolution is crystal clear and the smart features work flawlessly. Setup was easy and intuitive. Highly recommend for anyone looking for a great TV at this price point.
2. Rating: 4/5 - Great TV overall but the sound could be better. I ended up getting a soundbar. Picture is fantastic though!

Please provide your analysis in the following format:

1. **Overall Sentiment**: (Positive/Negative/Mixed) with percentage breakdown
2. **Common Praise**: Top 3 features customers love
3. **Common Complaints**: Top 3 issues or concerns
4. **Product Team Recommendations**: 2-3 specific actionable improvements
5. **Marketing Opportunities**: Which positive aspects

## Exercise 2 Solution: Zero-Shot Support Ticket Classification

In [3]:
ticket = walmart_data['customer_support_tickets'][0]

# SOLUTION: Zero-shot prompt with clear criteria
zero_shot_ticket_prompt = f"""You are a customer support ticket triage specialist.

Classify the priority of this support ticket based on these criteria:

Priority Levels:
- CRITICAL: Product safety issues, data breaches, or legal concerns
- HIGH: Product defects, significant customer impact, time-sensitive issues
- MEDIUM: Shipping delays, minor product issues, general inquiries
- LOW: Questions, feature requests, non-urgent feedback

Support Ticket:
Type: {ticket['issue_type']}
Description: {ticket['description']}
Current Status: {ticket['status']}

Provide:
1. Priority Level: (CRITICAL/HIGH/MEDIUM/LOW)
2. Reasoning: (2-3 sentences explaining the classification)
3. Suggested Response Time: (e.g., within 1 hour, within 24 hours, within 3 days)
"""

print("Zero-Shot Ticket Classification:")
print(get_completion(zero_shot_ticket_prompt, temperature=0))

Zero-Shot Ticket Classification:
1. **Priority Level:** HIGH  
2. **Reasoning:** The issue involves a product defect, which can significantly impact the customer's experience with the product. While a dead pixel is not a safety concern, it is a noticeable defect that may require repair or replacement, making it a time-sensitive issue to maintain customer satisfaction.  
3. **Suggested Response Time:** Within 24 hours  


## Exercise 3 Solution: Few-Shot Product Description Writer

In [4]:
# SOLUTION: Few-shot prompt with examples demonstrating desired style
few_shot_description_prompt = """You are writing engaging product descriptions for Walmart's website.
Follow the style and structure shown in these examples:

Example 1:
Product: Samsung 55-inch 4K Smart TV
Description: Transform your living room into a home theater! 🎬 This stunning Samsung 4K Smart TV delivers crystal-clear picture quality with vibrant HDR colors that bring every scene to life. Stream your favorite shows effortlessly with built-in apps, control everything with your voice, and enjoy the sleek, modern design that complements any décor. Perfect for movie nights, sports, and gaming!

Example 2:
Product: Instant Pot Duo 7-in-1
Description: Say goodbye to kitchen chaos! 👨‍🍳 The Instant Pot Duo replaces 7 appliances in one compact design—pressure cook, slow cook, steam, sauté, and more! Cut cooking time in half while keeping all the flavor. From weeknight dinners to meal prep Sundays, this kitchen powerhouse makes healthy, delicious meals easier than ever. Your family will wonder what took you so long!

Now write a description for this product following the same style:
Product: {}
Key Features: {}
Price: ${}
Description:"""

# Test with LEGO set
test_product = walmart_data['products'][5]
prompt = few_shot_description_prompt.format(
    test_product['name'],
    test_product['description'],
    test_product['price']
)

print("Few-Shot Product Description:")
print(get_completion(prompt, temperature=0.7))

Few-Shot Product Description:
Embark on an intergalactic adventure! 🚀 The LEGO Star Wars Millennium Falcon is the ultimate build for fans of the galaxy far, far away. With 1,351 detailed pieces, recreate the iconic spaceship that’s ready to outrun Imperial Star Destroyers. Bring the saga to life with minifigures of Han Solo, Chewbacca, and crew—perfect for epic battles or display-worthy scenes. Whether you’re a Jedi Master or a Padawan, this set is a must-have for collectors and builders alike. May the Force be with you! 🌌


## Exercise 4 Solution: Product Comparison Prompt Template

In [5]:
# SOLUTION: Comprehensive product comparison template
comparison_template = PromptTemplate(
    input_variables=["product1_name", "product1_price", "product1_desc", "product1_rating",
                    "product2_name", "product2_price", "product2_desc", "product2_rating",
                    "criteria", "target_audience"],
    template="""You are a product comparison expert helping Walmart customers make informed decisions.

Compare these two products for {target_audience}:

**Product A:**
- Name: {product1_name}
- Price: ${product1_price}
- Description: {product1_desc}
- Rating: {product1_rating}/5

**Product B:**
- Name: {product2_name}
- Price: ${product2_price}
- Description: {product2_desc}
- Rating: {product2_rating}/5

Comparison Criteria: {criteria}

Provide a detailed comparison that includes:
1. **Head-to-Head**: Compare both products across each criterion
2. **Best For**: Who should buy Product A vs Product B
3. **Value Analysis**: Which offers better value and why
4. **Final Recommendation**: Your top pick with clear reasoning

Format your response with clear headings and bullet points.
    """
)

# Test with Samsung TV vs AirPods (different categories for interesting comparison)
product1 = walmart_data['products'][0]  # Samsung TV
product2 = walmart_data['products'][7]  # AirPods

prompt = comparison_template.format(
    product1_name=product1['name'],
    product1_price=product1['price'],
    product1_desc=product1['description'],
    product1_rating=product1['rating'],
    product2_name=product2['name'],
    product2_price=product2['price'],
    product2_desc=product2['description'],
    product2_rating=product2['rating'],
    criteria="entertainment value, features, customer reviews, price-to-value ratio",
    target_audience="tech enthusiasts on a $300 budget"
)

print("Product Comparison Report:")
print(get_completion(prompt, temperature=0.5))

Product Comparison Report:
### **Head-to-Head Comparison**

#### 1. **Entertainment Value**
- **Product A: Samsung 55-inch 4K Smart TV**
  - Offers a large 55-inch 4K UHD display, ideal for immersive movie nights, gaming, and streaming.
  - HDR support enhances picture quality with vivid colors and deep contrasts.
  - Built-in streaming apps (e.g., Netflix, Hulu, Disney+) provide easy access to entertainment.
  - Voice control adds convenience for navigating content.
  - **Ideal for group entertainment or a home theater experience.**

- **Product B: Apple AirPods Pro (2nd Gen)**
  - Provides a premium audio experience with active noise cancellation (ANC) for immersive listening, whether for music, podcasts, or calls.
  - Transparency mode lets you stay aware of your surroundings while enjoying your content.
  - Personalized spatial audio delivers a 3D sound experience, especially great for audiophiles.
  - **Best for personal entertainment on the go.**

**Winner**: **Product A** for br

## Exercise 5 Solution: Choosing the Right Parameters

In [6]:
# Scenario 1 SOLUTION: Extract product prices from customer support tickets
ticket = walmart_data['customer_support_tickets'][0]

extraction_prompt = f"""Extract the product ID from this support ticket. Return only the product ID.

Ticket Description: {ticket['description']}
Product ID mentioned: {ticket['product_id']}

Product ID:"""

# SOLUTION: Low temperature for factual extraction
temperature = 0.0  # Deterministic, factual output
top_p = 1.0  # Not adjusting top_p when using temperature

print("Scenario 1: Data Extraction")
print(f"Temperature: {temperature} (Deterministic - need exact, consistent extraction)")
print(f"Top_p: {top_p} (Default)\n")

# Test multiple times to show consistency
print("Testing consistency (should be identical):")
for i in range(3):
    response = get_completion_with_top_p(extraction_prompt, top_p=top_p, temperature=temperature)
    print(f"  Attempt {i+1}: {response}")

Scenario 1: Data Extraction
Temperature: 0.0 (Deterministic - need exact, consistent extraction)
Top_p: 1.0 (Default)

Testing consistency (should be identical):
  Attempt 1: P001
  Attempt 2: P001
  Attempt 3: P001


In [7]:
# Scenario 2 SOLUTION: Generate creative product bundle ideas
products = walmart_data['products'][:3]

bundle_prompt = f"""Create 3 creative product bundle ideas combining these products:
1. {products[0]['name']} - ${products[0]['price']}
2. {products[1]['name']} - ${products[1]['price']}
3. {products[2]['name']} - ${products[2]['price']}

For each bundle:
- Give it a catchy, memorable name
- Explain the value proposition
- Suggest a bundle price with discount
- Identify the target customer
"""

# SOLUTION: High temperature for creative brainstorming
temperature = 0.9  # High creativity for diverse bundle ideas
top_p = 0.95  # Slightly focused to maintain coherence

print("Scenario 2: Creative Bundle Generation")
print(f"Temperature: {temperature} (High creativity - need diverse, innovative ideas)")
print(f"Top_p: {top_p} (Diverse but coherent)\n")

# Generate multiple creative variations
print("Generated Bundle Ideas (3 different creative outputs):")
for i in range(3):
    print(f"\n{'='*80}")
    print(f"Variation {i+1}:")
    print('='*80)
    response = get_completion_with_top_p(bundle_prompt, top_p=top_p, temperature=temperature)
    print(response)

Scenario 2: Creative Bundle Generation
Temperature: 0.9 (High creativity - need diverse, innovative ideas)
Top_p: 0.95 (Diverse but coherent)

Generated Bundle Ideas (3 different creative outputs):

Variation 1:
Here are three creative product bundle ideas combining the listed products:

---

### 1. **"Run, Watch, Refuel" Bundle**
- **Value Proposition:** This bundle is all about fueling your body, mind, and leisure. Enjoy the latest Nike Air Max shoes to power your runs, wind down by streaming your favorite shows on the Samsung 55-inch 4K Smart TV, and replenish your body with organic whole milk. It's the ultimate lifestyle package for health-conscious entertainment lovers.
- **Bundle Price:** $649.99 (Save $25 off the total price of $675.97)
- **Target Customer:** Active professionals or fitness enthusiasts who want to balance a healthy lifestyle with at-home relaxation.

---

### 2. **"Couch-to-5K Starter Kit"**
- **Value Proposition:** This bundle is perfect for anyone starting the

## Bonus: Advanced Real-World Application

In [9]:
# BONUS SOLUTION: Complete automated review response system

class ReviewResponseSystem:
    """Automated system for responding to customer reviews"""
    
    def __init__(self):
        # Initialize Azure OpenAI client using LangChain
        self.client = AzureChatOpenAI(
            azure_endpoint="https://aoi-ext-eus-aiml-profx-01.openai.azure.com/",
            api_key=os.environ["AZURE_OPENAI_API_KEY"],
            model="gpt-4o",
            api_version="2024-12-01-preview",
            temperature=0.5,
            top_p=1.0,
        )
        
        # Few-shot template for responses
        self.examples = [
            {
                "rating": "5",
                "review": "Love this product! Works perfectly and exceeded my expectations.",
                "response": "Thank you so much for your wonderful review! We're thrilled to hear our product exceeded your expectations. Your satisfaction means the world to us, and we appreciate you taking the time to share your experience. Happy shopping!"
            },
            {
                "rating": "3",
                "review": "Product is okay but the delivery was late.",
                "response": "Thank you for your honest feedback. We're glad the product meets your needs, but we sincerely apologize for the delivery delay. This isn't the experience we want for our customers. We're working to improve our delivery times and would love to make this right with a discount on your next order."
            },
            {
                "rating": "1",
                "review": "Terrible quality. Broke after one week.",
                "response": "We're extremely sorry to hear about your experience. This is absolutely not acceptable, and we want to make this right immediately. Please contact our support team at 1-800-WALMART or DM us your order number, and we'll send you a replacement with expedited shipping at no cost, plus a full refund. Your satisfaction is our priority."
            }
        ]
    
    def generate_response(self, review_text, rating):
        """Generate appropriate response based on review sentiment"""
        
        # Build few-shot prompt
        examples_text = "\n\n".join([
            f"Review (Rating {ex['rating']}/5): {ex['review']}\nResponse: {ex['response']}"
            for ex in self.examples
        ])
        
        prompt = f"""You are Walmart's customer service representative responding to product reviews.
Follow these examples to maintain consistent tone and approach:

{examples_text}

Now generate a response for:
Review (Rating {rating}/5): {review_text}
Response:"""
        
        # Use the Azure OpenAI client to generate response
        response = self.client.invoke(prompt)
        
        return response.content

# Test the system
system = ReviewResponseSystem()

print("Automated Review Response System\n")
print("="*80)

for i, review in enumerate(walmart_data['customer_reviews'][:3], 1):
    print(f"\nReview {i}:")
    print(f"Rating: {review['rating']}/5")
    print(f"Review: {review['review_text']}")
    print(f"\nGenerated Response:")
    response = system.generate_response(review['review_text'], review['rating'])
    print(response)
    print("\n" + "="*80)

Automated Review Response System


Review 1:
Rating: 5/5
Review: Amazing picture quality! The 4K resolution is crystal clear and the smart features work flawlessly. Setup was easy and intuitive. Highly recommend for anyone looking for a great TV at this price point.

Generated Response:
Thank you so much for your fantastic review! We're delighted to hear that you're enjoying the amazing picture quality and seamless smart features of your new TV. It's wonderful to know the setup process was easy and intuitive for you. Your recommendation means a lot to us, and we're thrilled to offer a product that exceeds your expectations at a great value. Happy watching!


Review 2:
Rating: 4/5
Review: Great TV overall but the sound could be better. I ended up getting a soundbar. Picture is fantastic though!

Generated Response:
Thank you so much for your fantastic review! We're delighted to hear that you're enjoying the amazing picture quality and seamless smart features of your new TV. It's wonderf

## Summary of Solutions

### Key Principles Applied:

1. **Exercise 1 (Prompt Engineering)**:
   - Added clear role definition
   - Structured output format
   - Specific, actionable requirements

2. **Exercise 2 (Zero-Shot)**:
   - Clear criteria definition
   - Structured output format
   - Temperature 0 for consistency

3. **Exercise 3 (Few-Shot)**:
   - Quality examples showing desired style
   - Consistent format across examples
   - Medium temperature for creativity with structure

4. **Exercise 4 (Templates)**:
   - Comprehensive variable coverage
   - Reusable structure
   - Clear formatting instructions

5. **Exercise 5 (Temperature/Top_p)**:
   - Scenario 1: Temp 0.0 for factual extraction
   - Scenario 2: Temp 0.9 for creative ideation
   - Proper justification for each choice

🎉 **Congratulations on completing Module 02!**